In [4]:
# import cv2
# import numpy as np

# # Đường dẫn đến file mô hình và cấu hình
# MODEL_PATH = "MobileNetSSD_deploy.caffemodel"
# CONFIG_PATH = "MobileNetSSD_deploy.prototxt"
# LABELS_PATH = "labels.txt"

# # Danh sách nhãn của các đối tượng
# CLASSES = []
# with open(LABELS_PATH, 'r') as f:
#     CLASSES = [line.strip() for line in f.readlines()]

# # Tải mô hình MobileNet SSD
# net = cv2.dnn.readNetFromCaffe(CONFIG_PATH, MODEL_PATH)

# # Khởi tạo webcam
# cap = cv2.VideoCapture(0)
# cap.set(cv2.CAP_PROP_FRAME_WIDTH, 720)  # Giảm độ phân giải để tăng tốc
# cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

# while True:
#     ret, frame = cap.read()
#     if not ret:
#         break

#     # Chuẩn bị frame cho mô hình
#     (h, w) = frame.shape[:2]
#     blob = cv2.dnn.blobFromImage(frame, 0.007843, (300, 300), 127.5)

#     # Đưa blob vào mô hình
#     net.setInput(blob)
#     detections = net.forward()

#     # Xử lý kết quả nhận diện
#     for i in range(detections.shape[2]):
#         confidence = detections[0, 0, i, 2]
#         if confidence > 0.5:  # Ngưỡng tin cậy
#             idx = int(detections[0, 0, i, 1])
#             box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
#             (startX, startY, endX, endY) = box.astype("int")

#             # Vẽ hộp và nhãn
#             label = f"{CLASSES[idx]}: {confidence:.2f}"
#             cv2.rectangle(frame, (startX, startY), (endX, endY), (0, 255, 0), 2)
#             cv2.putText(frame, label, (startX, startY - 10),
#                         cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

#     # Hiển thị frame
#     cv2.imshow("Object Detection", frame)

#     # Nhấn 'q' để thoát
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break

# # Giải phóng tài nguyên
# cap.release()
# cv2.destroyAllWindows()

In [ ]:
import cv2
import numpy as np
import time
from datetime import datetime
import logging
from scipy.spatial import distance

# Cấu hình logging
logging.basicConfig(filename='object_detection.log', level=logging.INFO,
                    format='%(asctime)s - %(message)s')

# Đường dẫn đến file mô hình và cấu hình
MODEL_PATH = "MobileNetSSD_deploy.caffemodel"
CONFIG_PATH = "MobileNetSSD_deploy.prototxt"
LABELS_PATH = "labels.txt"

# Danh sách nhãn của các đối tượng
CLASSES = []
with open(LABELS_PATH, 'r') as f:
    CLASSES = [line.strip() for line in f.readlines()]

# Tải mô hình MobileNet SSD
net = cv2.dnn.readNetFromCaffe(CONFIG_PATH, MODEL_PATH)

# Khởi tạo webcam
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 720)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

# Biến để theo dõi đối tượng và log
tracked_objects = {}  # Lưu trữ đối tượng: {id: {'class', 'center', 'box', 'confidence'}}
next_id = 0  # ID tiếp theo để gán cho đối tượng mới
last_log_time = time.time()  # Thời gian log định kỳ cuối cùng
LOG_INTERVAL = 10  # Ghi log mỗi 10 giây
MOVE_THRESHOLD = 360  # Ngưỡng di chuyển (nửa màn hình: 720/2 = 360 pixel)
FRAME_WIDTH = 720  # Chiều rộng khung hình

def assign_objects(current_objects):
    global next_id, tracked_objects
    new_tracked_objects = {}
    used_ids = set()

    # Gán đối tượng hiện tại cho các đối tượng đã theo dõi
    for curr_obj in current_objects:
        curr_center = curr_obj['center']
        min_dist = float('inf')
        matched_id = None

        # Tìm đối tượng trước đó gần nhất
        for obj_id, prev_obj in tracked_objects.items():
            if obj_id in used_ids:
                continue
            dist = distance.euclidean(curr_center, prev_obj['center'])
            if dist < min_dist and dist < 100:  # Ngưỡng khoảng cách để gán cùng đối tượng
                min_dist = dist
                matched_id = obj_id

        # Nếu tìm thấy đối tượng khớp
        if matched_id is not None:
            new_tracked_objects[matched_id] = curr_obj
            used_ids.add(matched_id)
            # Kiểm tra di chuyển đáng kể
            prev_center = tracked_objects[matched_id]['center']
            move_dist = distance.euclidean(curr_center, prev_center)
            if move_dist > MOVE_THRESHOLD:
                logging.info(f"Object {matched_id} ({curr_obj['class']}) moved significantly: "
                             f"{move_dist:.2f} pixels")
                print(f"Object {matched_id} ({curr_obj['class']}) moved {move_dist:.2f} pixels")
        else:
            # Gán ID mới cho đối tượng mới
            new_tracked_objects[next_id] = curr_obj
            logging.info(f"New object detected: ID {next_id}, Class: {curr_obj['class']}, "
                         f"Confidence: {curr_obj['confidence']:.2f}, Center: {curr_obj['center']}")
            print(f"New object: ID {next_id}, Class: {curr_obj['class']}")
            next_id += 1

    # Kiểm tra đối tượng biến mất
    for obj_id, prev_obj in tracked_objects.items():
        if obj_id not in new_tracked_objects:
            logging.info(f"Object {obj_id} ({prev_obj['class']}) disappeared")
            print(f"Object {obj_id} ({prev_obj['class']}) disappeared")

    tracked_objects = new_tracked_objects

def log_periodic():
    """Ghi log định kỳ trạng thái hiện tại"""
    log_message = f"Periodic log: {len(tracked_objects)} objects detected"
    for obj_id, obj in tracked_objects.items():
        log_message += f"\n- ID {obj_id}: {obj['class']}, Confidence: {obj['confidence']:.2f}, " \
                       f"Center: {obj['center']}"
    logging.info(log_message)
    print("\n=== Periodic Log ===")
    print(log_message)

frame_count = 0
start_time = time.time()

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Chuẩn bị frame cho mô hình
    (h, w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame, 0.007843, (300, 300), 127.5)

    # Đưa blob vào mô hình
    net.setInput(blob)
    detections = net.forward()

    # Lưu danh sách đối tượng hiện tại
    current_objects = []
    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > 0.5:  # Ngưỡng tin cậy
            idx = int(detections[0, 0, i, 1])
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")

            # Tính trung tâm và kích thước
            center_x = (startX + endX) // 2
            center_y = (startY + endY) // 2
            box_width = endX - startX
            box_height = endY - startY

            # Lưu thông tin đối tượng
            obj_info = {
                'class': CLASSES[idx],
                'confidence': confidence,
                'center': (center_x, center_y),
                'box': (startX, startY, endX, endY),
                'size': (box_width, box_height)
            }
            current_objects.append(obj_info)

            # Vẽ hộp và nhãn
            label = f"{CLASSES[idx]}: {confidence:.2f}"
            cv2.rectangle(frame, (startX, startY), (endX, endY), (0, 255, 0), 2)
            cv2.putText(frame, label, (startX, startY - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Gán và theo dõi đối tượng
    assign_objects(current_objects)

    # Kiểm tra log định kỳ (mỗi 10 giây)
    current_time = time.time()
    if current_time - last_log_time >= LOG_INTERVAL:
        log_periodic()
        last_log_time = current_time

    # Tính FPS
    frame_count += 1
    elapsed_time = current_time - start_time
    fps = frame_count / elapsed_time if elapsed_time > 0 else 0

    # Hiển thị FPS trên frame
    cv2.putText(frame, f"FPS: {fps:.2f}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # Hiển thị frame
    cv2.imshow("Object Detection", frame)

    # Nhấn 'q' để thoát
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Giải phóng tài nguyên
cap.release()
cv2.destroyAllWindows()

New object: ID 0, Class: person
New object: ID 1, Class: person
New object: ID 2, Class: person
Object 2 (person) disappeared
New object: ID 3, Class: person
Object 3 (person) disappeared
Object 0 (person) disappeared
New object: ID 4, Class: person
Object 4 (person) disappeared
New object: ID 5, Class: person
New object: ID 6, Class: person
Object 6 (person) disappeared
New object: ID 7, Class: chair
Object 7 (chair) disappeared
New object: ID 8, Class: chair
Object 8 (chair) disappeared
New object: ID 9, Class: chair
Object 9 (chair) disappeared
New object: ID 10, Class: bottle
Object 10 (bottle) disappeared
New object: ID 11, Class: bottle
Object 11 (bottle) disappeared
New object: ID 12, Class: person
Object 5 (person) disappeared
New object: ID 13, Class: person
Object 12 (person) disappeared
New object: ID 14, Class: person
Object 13 (person) disappeared
New object: ID 15, Class: person
Object 14 (person) disappeared
New object: ID 16, Class: person
Object 16 (person) disappeared